In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
import langchain
from itertools import product
import pandas as pd
import os

md_data_dir = os.path.join("dataset", "markdown")



def cut_and_clean_file(filename):
    file = os.path.join(md_data_dir, filename)
    with open(file, encoding="utf-8") as f:
        text = f.read()

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]


    point_bullets_second = [(f"\n{i}.{j}. ", f"Пункт {i}.{j}") for i,j in product(range(1,9), range(1,50))]
    point_bullets_third = [(f"\n{i}.{j}.{k}. ", f"Пункт {i}.{j}.{k}") for i,j,k in product(range(1,9), range(1,50), range(1,50))]

    for splitter, name in point_bullets_second:
        text=text.replace(splitter, "\n## " + name + "\n")

    for splitter, name in point_bullets_third:
        text=text.replace(splitter, "\n### " + name + "\n")


    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on, strip_headers=True)
    md_header_splits = markdown_splitter.split_text(text)

    num_of_headers = len(headers_to_split_on)
    pages = []
    headers = [[] for _ in range(num_of_headers)]


    for doc in md_header_splits:
        meta = doc.metadata
        content = doc.page_content
        pages.append(content)
        for header in range(num_of_headers):
            headers[header].append(meta.get(f"Header {header + 1}", None))

    result_dict = {f"Header_{i + 1}": h for i, h in enumerate(headers)}
    result_dict["content"] = pages
    pd.DataFrame(result_dict).to_csv(f"dataset/clean/{filename[:-3]}.csv", index=False)

In [160]:
files = os.listdir(md_data_dir)

for file in files:
    cut_and_clean_file(file)